In [1]:
import pyodbc
import pandas as pd

In [2]:
pyodbc.drivers()

['SQL Server',
 'Microsoft Access Driver (*.mdb, *.accdb)',
 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)',
 'Microsoft Access Text Driver (*.txt, *.csv)',
 'SQL Server Native Client RDA 11.0',
 'ODBC Driver 17 for SQL Server']

In [3]:
conn = pyodbc.connect(
    Trusted_Connection = "Yes",
    Driver = '{ODBC Driver 17 for SQL Server}',
    Server = 'DESKTOP-SNJIG4N',
    Database = "Transporte_Público_en_España"
)
cursor = conn.cursor()

In [4]:
df = pd.read_csv('./data/Transporte_Publico_Espana.csv')

In [5]:
df.columns

Index(['ID_Viaje', 'Fecha', 'Ruta', 'Numero_Pasajeros',
       'Duracion_Viaje_Minutos', 'Retraso_Minutos', 'Tipo_Transporte',
       'Region', 'Dia_Semana'],
      dtype='object')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100100 entries, 0 to 100099
Data columns (total 9 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   ID_Viaje                100100 non-null  int64  
 1   Fecha                   100100 non-null  object 
 2   Ruta                    100100 non-null  object 
 3   Numero_Pasajeros        99099 non-null   float64
 4   Duracion_Viaje_Minutos  99767 non-null   object 
 5   Retraso_Minutos         100100 non-null  int64  
 6   Tipo_Transporte         100100 non-null  object 
 7   Region                  100100 non-null  object 
 8   Dia_Semana              100100 non-null  object 
dtypes: float64(1), int64(2), object(6)
memory usage: 6.9+ MB


In [7]:
df.head()

,ID_Viaje,Fecha,Ruta,Numero_Pasajeros,Duracion_Viaje_Minutos,Retraso_Minutos,Tipo_Transporte,Region,Dia_Semana
0,1,2023-01-01 00:00:00,Barcelona - Bilbao,NaN,not_available,999,Tranvía,Sevilla,Sunday
1,2,2023-01-01 00:01:00,Malaga - Sevilla,99.0,192.0,58,Tren,Barcelona,Sunday
2,3,2023-01-01 00:02:00,Madrid - Zaragoza,10.0,94.0,119,Autobús,Madrid,Sunday
3,4,2023-01-01 00:03:00,Sevilla - Madrid,96.0,650.0,2,Metro,Malaga,Sunday
4,5,2023-01-01 00:04:00,Madrid - Malaga,22.0,335.0,51,Autobús,Sevilla,Sunday


In [8]:
#Conversión de tipos de datos
df['Numero_Pasajeros'] = pd.to_numeric(df['Numero_Pasajeros'], errors='coerce')
df['Duracion_Viaje_Minutos'] = pd.to_numeric(df['Duracion_Viaje_Minutos'], errors='coerce')

# Imputación de valores faltantes en Numero_Pasajeros y Duracion_Viaje_Minutos
df['Numero_Pasajeros'] = df['Numero_Pasajeros'].fillna(df['Numero_Pasajeros'].median())
df['Duracion_Viaje_Minutos'] = df['Duracion_Viaje_Minutos'].fillna(df['Duracion_Viaje_Minutos'].median())

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100100 entries, 0 to 100099
Data columns (total 9 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   ID_Viaje                100100 non-null  int64  
 1   Fecha                   100100 non-null  object 
 2   Ruta                    100100 non-null  object 
 3   Numero_Pasajeros        100100 non-null  float64
 4   Duracion_Viaje_Minutos  100100 non-null  float64
 5   Retraso_Minutos         100100 non-null  int64  
 6   Tipo_Transporte         100100 non-null  object 
 7   Region                  100100 non-null  object 
 8   Dia_Semana              100100 non-null  object 
dtypes: float64(2), int64(2), object(5)
memory usage: 6.9+ MB


In [9]:
cursor.execute("""
CREATE TABLE Transporte_Publico (
    ID_Viaje int,
    Fecha nvarchar(50),
    Ruta nvarchar(50),
    Numero_Pasajeros float,
    Duracion_Viaje_Minutos float,
    Retraso_Minutos int,
    Tipo_Transporte nvarchar(50),
    Region nvarchar(50),
    Dia_Semana nvarchar(50)
)
""")


In [10]:
for row in df.itertuples():
    cursor.execute("""
                INSERT INTO Transporte_Público_en_España.dbo.Transporte_Publico(ID_Viaje, Fecha,Ruta, Numero_Pasajeros,
                Duracion_Viaje_Minutos, Retraso_Minutos, Tipo_Transporte, Region, Dia_Semana)
                VALUES(?,?,?,?,?,?,?,?,?)
                """,
                row.ID_Viaje, 
                row.Fecha,
                row.Ruta,
                row.Numero_Pasajeros,
                row.Duracion_Viaje_Minutos,
                row.Retraso_Minutos, 
                row.Tipo_Transporte,
                row.Region,
                row.Dia_Semana)
conn.commit()